# Programmieraufgabe 1: (gedämpftes) Newton-Verfahren

**Abgabe in den Programmiertutorien am 14. und 15. Mai 2025.**

Benötigte Module für dieses Notebook:

In [66]:
import numpy as np
import numpy.linalg as la

In diesem Notebook wollen wir das Newton-Verfahren und das gedämpfte Newton-Verfahren zur Approximation einer Nullstelle der Funktion
$$f: \mathbb{R}^2 \to \mathbb{R}^2, \quad x = (x_1,x_2) \mapsto \begin{pmatrix} 1 - \frac{2}{\exp(x_1-x_2)+1} \\ x_2^3 + x_2 -2 \end{pmatrix}$$
testen.

**(a) Schreiben Sie jeweils eine Prozedur, die für einen Vektor $x\in\mathbb{R}^2$ die Funktion bzw. deren Ableitung an der Stelle $x$ berechnet und als `numpy`-array zurückgibt.**

In [138]:
def f(v):
    x = 1 - 2/(np.exp(v[0] - v[1]) +1)
    y = v[1] ** 3 + v[1] - 2
    return np.array([x, y])

In [139]:
def df(v):
    x = v[0]
    y = v[1]
    e = np.exp(x - y)
    denom = (e + 1) ** 2
    res = np.zeros((2, 2))
    res[0, 0] = 2 * e / denom
    res[0, 1] = -2 * e / denom
    res[1, 0] = 0
    res[1, 1] = 3 * y ** 2 + 1   
    return res

In [140]:
f([10, 1])
df([10, 1])


array([[ 2.467587e-04, -2.467587e-04],
       [ 0.000000e+00,  4.000000e+00]])

**(b) Schreiben Sie eine Prozedur, die das Newton-Verfahren mit den in der Vorlesung besprochenen Kovergenz- und Abbruchkriterien auf eine Funktion $f:\mathbb{R}^n \to \mathbb{R}^n$ für beliebiges $n\in\mathbb{N}$ anwendet.**

Ihrer Prozedur soll folgende Eingabedaten haben:
- Einen Vektor `x0`, der den Startwert $x^{(0)}\in\mathbb{R}^n$ für das Newton-Verfahren enthält.
- Zwei Prozeduren `f` und `fprime`, mit denen Funktions- und Ableitungswerte von $f$ berechnet werden können.
- Eine Toleranz `tol` und eine maximale Zahl an Iterationen `kMax`, mit denen die Konvergenz- und Abbruchkriterien gesteuert werden können.

Berücksichtigen Sie außerdem folgendes:
- Die im Newton-Verfahren auftretenden Gleichungssysteme können Sie mit dem in `numpy` enthaltenen LGS-Löser `np.linalg.solve` lösen.
- Am Ende der Prozedur sollen alle berechneten Iterierten $x^{(k)}$, $k=0,1,2,...$ an das Hauptprogramm zurückgegeben werden. Geben Sie außerdem eine Meldung aus, ob das Verfahren erfolgreich konvergiert ist, oder ob es wegen Divergenz bzw. wegen Erreichen der maximalen Iterationszahl abgebrochen hat.

_Hinweis:_ Sie können alle Iterierten zum Beispiel als Zeilen in einer gemeinsamen Matrix speichern. Jedes Mal, wenn eine neue Iterierte berechnet wurde, wird die Matrix dementsprechend um eine Zeile ergänzt. Dazu ist der Befehl `np.vstack` hilfreich. Beispielsweise werden drei Matrizen `A1,A2,A3` derselben Breite mit dem Aufruf `np.vstack((A1,A2,A3))` übereinander gestapelt.

In [ ]:
def newton (x0, h, hprime, tol, kMax):
    iterations = np.array(x0).reshape(1, -1)
    iteration = 0
    error = tol + 1
    error_delta_before = None
    while error >= tol:
        iterated = iterations[-1, :]
        position = h(iterated)
        speed = hprime(iterated)
        try:
            delta = la.solve(speed, -position)
        except la.LinAlgError:
            print("Abbruch: Jacobimatrix ist singulär.")
            break
        
        new_iterated = iterated + delta
        iterations = np.vstack([iterations, new_iterated])
        iteration += 1
        error = la.norm(delta)

        error_delta = la.solve(speed, h(new_iterated))
        if error_delta_before is not None and la.norm(error_delta) > la.norm(error_delta_before):
            print("Abbruch: Fehler wird größer: " + str(la.norm(error_delta)) + " > " + str(la.norm(error_delta_before)))
            break
        error_delta_before = error_delta

        if iteration >= kMax:
            print("Abbruch: Zu viele Iterationen")
            break
    return iterations

**(c) Testen Sie Ihre Prozedur für die obige Funktion $f$ mit den Startvektoren $x^{(0)} = (4,2)^T$ und $x^{(0)} = (4,-4)^T$. Verwenden Sie die Parameter `tol = 1e-8` und `kMax = 20`.**

Geben Sie alle Iterierten aus. Für welchen Startvektor konvergiert das Verfahren? Vergewissern Sie sich im Falle der Konvergenz anhand der Funktionsvorschrift von $f$, dass Sie tatsächlich eine Nullstelle von $f$ erhalten haben. Geben Sie in dem Fall auch den Fehler der einzelnen Iterierten aus. Können Sie die erwartete, quadratische Konvergenz erkennen?

Startwert $x^{(0)} = (4,2)^T$:

In [142]:
newton([4,2], f, df, 1e-8, 20)

array([[ 4.        ,  2.        ],
       [-0.24224502,  1.38461538],
       [ 1.90139076,  1.08258613],
       [ 0.91017052,  1.00478035],
       [ 1.00015828,  1.00001707],
       [ 1.        ,  1.        ],
       [ 1.        ,  1.        ]])

Startwert $x^{(0)} = (4,-4)^T$:

In [143]:
newton([4,-4], f, df, 1e-8, 20)

Abbruch: Jacobimatrix ist singulär.


array([[    4.        ,    -4.        ],
       [-1485.05025436,    -2.57142857]])

**(d) Kopieren Sie das Newton-Verfahren von oben und ändern Sie es zum gedämpften Newton-Verfahren ab. Der Parameter $\lambda_{\min}$ soll dabei als zusätzlicher Eingabeparameter übergeben werden, während im ersten Schritt immer mit dem Wert $\lambda = 1$ gestartet wird. Geben Sie zusätzlich zu den Iterierten auch die verwendeten Dämpungsparameter an das Hauptprogramm zurück.**

_Hinweis:_ Das Schlüsselwort `lambda` hat in Python eine ganz besondere Bedeutung und sollte daher nicht als Variablenname genutzt werden.

In [172]:
# In manchen Zeilen nimmt ein kleines Lämmlein den Bus.
# Wohin es wohl reist? Vielleicht in den Karateclub von Augsburg?
# Ich habe gehört, da kann er was für seine Kondition machen (haha Mathewitz)

def dampened_newton (x0, h, hprime, tol, kMax, lammbus_min): #🐑🚌
    iterations = np.array(x0).reshape(1, -1)
    iteration = 0
    error = tol + 1
    lammbus = 1 #🐑🚌
    while error >= tol and iteration < kMax:
        iterated = iterations[-1, :]
        position = h(iterated)
        speed = hprime(iterated)
        delta = None
        try:
            delta = la.solve(speed, -position)
        except la.LinAlgError:
            print("Abbruch: Jacobimatrix ist singulär.")
            break


        lammbus = 1.0 #🐑🚌
        while la.norm(h(iterated + lammbus * delta)) > (1 - lammbus / 2) * la.norm(position): #🐑🚌
            print (lammbus)
            lammbus /= 2 #🐑🚌
            if lammbus < lammbus_min: #🐑🚌
                print("Abbruch: Minimale Dämpfung unterschritten.")
                return iterations

        new_iterated = iterated + lammbus * delta
        iterations = np.vstack([iterations, new_iterated])
        error = la.norm(h(new_iterated))
        iteration += 1
    return iterations

**(e) Wiederholen Sie Teil (c) mit dem gedämpften Newton-Verfahren. Verwenden Sie dabei $\lambda_{\min}=10^{-10}$ und geben Sie die Werte des Dämpungsparameters aus.**

Startwert $x^{(0)} = (4,2)^T$:

In [173]:
dampened_newton([4,2], f, df, 1e-8, 20, 10**(-10))

array([[ 4.        ,  2.        ],
       [-0.24224502,  1.38461538],
       [ 1.90139076,  1.08258613],
       [ 0.91017052,  1.00478035],
       [ 1.00015828,  1.00001707],
       [ 1.        ,  1.        ]])

Startwert $x^{(0)} = (4,-4)^T$:

In [174]:
dampened_newton([4,-4], f, df, 1e-8, 20, 10**(-10))

Abbruch: Jacobimatrix ist singulär.


array([[    4.        ,    -4.        ],
       [-1485.05025436,    -2.57142857]])